# Загрузка данных и обработка

## Импорт и загрузка

In [1]:
import pandas as pd

In [2]:
# Убирает пробелы при загрузке датасета
def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text
# Понадобилось самому задавать имена колонок, т.к. считывались с пробелами
# К тому же, пришлось сделать strip на все коорд. данные, они (вообще все) данные идут с лишними пробелами
# У вас должен быть файл checkins.dat в текущей директории!
try:
    raw_data = pd.read_csv('checkins.dat', delimiter='|', 
                   names=['id', 'user_id', 'latitude', 'longitude', 'created_at'], header=0, 
                   converters={'latitude': strip, 'longitude': strip})
except IOError:
    print "Не удалось открыть файл checkins.dat! Вы уверены, что он есть в текущем каталоге?"
    print "Дальнейший код запускать не имеет смысла без этого файла."

/home/aleksei/.virtualenvs/python2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw_data.head()

,id,user_id,latitude,longitude,created_at
---------+---------+----------+-------------------+-------------------+---------------------,NaN,NaN,,,NaN
984301,2041916.0,5222.0,,,2012-04-21 17:39:01
984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
984315,1764391.0,5222.0,,,2012-04-21 17:37:18
984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43


In [4]:
raw_data = raw_data.dropna()

In [5]:
raw_data.head()

,id,user_id,latitude,longitude,created_at
984301,2041916.0,5222.0,,,2012-04-21 17:39:01
984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
984315,1764391.0,5222.0,,,2012-04-21 17:37:18
984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
984249,2146840.0,5222.0,,,2012-04-21 17:42:58


## Отсечение данных без координат 

In [6]:
ready_data = raw_data[(raw_data.latitude != '') & (raw_data.longitude != '')]

In [7]:
ready_data.head()

,id,user_id,latitude,longitude,created_at
984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [8]:
ready_data.shape

(396634, 5)

#### Готово. Теперь с датасетом можно работать

# Построение модели

In [9]:
from sklearn.cluster import MeanShift
import numpy as np

### Возмем только latitude и longitude

In [10]:
cluster_data = ready_data[['latitude', 'longitude']][:100*10**3]

### Строим и обучаем модель

In [11]:
model = MeanShift(bandwidth=0.1, n_jobs=4)

In [12]:
%time model.fit(cluster_data)

CPU times: user 22.6 s, sys: 1 s, total: 23.6 s
Wall time: 1min 27s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=4, seeds=None)

In [13]:
predictions = model.predict(cluster_data)

### Подсчитаем количество вхождений каждого кластера

In [14]:
unique, counts = np.unique(predictions, return_counts=True)
occurences = zip(unique, counts)
occurences = pd.DataFrame(counts, unique)

In [15]:
occurences.head()

,0
0,12506
1,4692
2,3994
3,3363
4,3526


### Добавим в датасет информацию о кластере, в который попадает объект

In [16]:
cluster_data['cluster'] = predictions

### Отсечем те места, кластеры которых содержат меньше $n$ точек

In [17]:
n = 15

In [18]:
cluster_data['rare_cluster'] = (occurences.iloc[cluster_data.cluster] < n).as_matrix()[:,0]

In [19]:
pop_cluster_data = cluster_data[cluster_data.rare_cluster == False]

In [20]:
pop_cluster_data.head()

,latitude,longitude,cluster,rare_cluster
984222,38.8951118,-77.0363658,5,False
984234,33.800745,-84.41052,7,False
984291,45.5234515,-122.6762071,30,False
984318,40.764462,-111.904565,66,False
984232,33.4483771,-112.0740373,1,False


In [21]:
pop_cluster_data.shape

(91351, 4)

In [22]:
model.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [  39.2190608 , -121.0610606 ],
       [  -4.5585849 ,  105.4068079 ],
       [  46.7312745 , -117.1796158 ]])

In [23]:
unique_pop_cluster_centers = model.cluster_centers_[np.unique(pop_cluster_data.cluster)]

In [24]:
unique_pop_cluster_centers[:10]

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       [  41.87824378,  -87.62984336],
       [  37.68868157, -122.40933037],
       [  38.88616522,  -77.04878333],
       [  33.35734456, -111.82265411],
       [  33.76663623,  -84.39328918],
       [  42.36321864,  -71.07368761],
       [  47.60624472, -122.33204383]])

In [25]:
unique_pop_cluster_centers.shape

(624, 2)

### Доходим до конца! Подсчитаем расстояние от центров кластеров до ближайшего офиса

In [26]:
offices = pd.DataFrame([(33.751277, -118.188740, 'Los Angeles'),
                        (25.867736, -80.324116,  'Miami'),
                        (51.503016, -0.075479, 'London'),
                        (52.378894, 4.885084, 'Amsterdam'),
                        (39.366487, 117.036146, 'Beijing'),
                        (-33.868457, 151.205134, 'Sydney')], columns=['latitude', 'longitude', 'City'])

In [27]:
offices

,latitude,longitude,City
0,33.751277,-118.188740,Los Angeles
1,25.867736,-80.324116,Miami
2,51.503016,-0.075479,London
3,52.378894,4.885084,Amsterdam
4,39.366487,117.036146,Beijing
5,-33.868457,151.205134,Sydney


In [28]:
unique_pop_cluster_centers_distances = []
upccd = []
for i in range(unique_pop_cluster_centers.shape[0]):
    tmp = []
    for coords in offices[['latitude', 'longitude']].as_matrix():
        tmp.append(np.linalg.norm(unique_pop_cluster_centers[i] - coords))
    upccd.append(np.min(tmp))

In [29]:
upccd = np.array(upccd)

#### Находим минимум

In [30]:
ans = unique_pop_cluster_centers[np.argmin(upccd)]

### Запишем ответ

In [31]:
with open('ans', 'w') as f:
    f.write("{} {}".format(*ans))